<a href="https://colab.research.google.com/github/kirikorneev/text2video/blob/main/text_to_video_synthesis_diffusers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/huggingface/diffusers/pull/2738 modified
!pip install git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg] -U
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-lj8zvgok
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-lj8zvgok
  Resolved https://github.com/huggingface/diffusers to commit 2de36fae7b15388ea44b8953ce60682adb3429b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.15.0.d

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode
import datetime

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

!mkdir /content/videos
!mkdir /content/videos/frames

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
import random
prompt = 'Thierry Baudet fighting Godzilla' #@param {type:"string"}
negative_prompt = "ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, signature, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur, distorted face" #@param {type:"string"}
num_frames = 32 #@param {type:"raw"}
seed = 27 #@param {type:"raw"}
if seed == None:
        seed = random.randint(0, 1000000)
generator = torch.Generator().manual_seed(seed)
video_frames = pipe(prompt, height=320, width=320, 
                    generator=generator, 
                    negative_prompt=negative_prompt, 
                    num_inference_steps=60, 
                    guidance_scale = 20, 
                    num_frames=num_frames).frames
output_video_path = export_to_video(video_frames)
prompt_name = prompt.replace(" ","_").lower()
print(prompt_name)

new_video_path = f'/content/videos/{prompt_name}-{datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")}.mp4'
!ffmpeg -y -i {output_video_path} -c:v libx264 -c:a aac -strict -2 {new_video_path} >/dev/null 2>&1
!ffmpeg -i {new_video_path} '/content/videos/frames/%04d.png' >/dev/null 2>&1

print(output_video_path, '->', new_video_path)
!cp {new_video_path} /content/videos/tmp.mp4
mp4 = open('/content/videos/tmp.mp4','rb').read()

decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{decoded_vid}" type="video/mp4"></video>')


  0%|          | 0/25 [00:00<?, ?it/s]

thierry_baudet_eating_a_cucumber
/tmp/tmp4w_gpf65.mp4 -> /content/videos/thierry_baudet_eating_a_cucumber-2023-04-07_09:35:15.mp4


In [ ]:
from google.colab import files
files.download(new_video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp {new_video_path} /content/videos/tmp.mp4
mp4 = open('/content/videos/tmp.mp4','rb').read()

decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{decoded_vid}" type="video/mp4"></video>')